<p style="text-align:center">
PSY 394U <b>Data Analytics with Python</b>, Spring 2018


<img style="width: 400px; padding: 0px;" src="https://github.com/sathayas/JupyterAnalyticsSpring2018/blob/master/images/Title_pics.png?raw=true" alt="title pics"/>

</p>

<p style="text-align:center; font-size:40px; margin-bottom: 30px;"><b> Network data basics </b></p>

<p style="text-align:center; font-size:18px; margin-bottom: 32px;"><b>March 22, 2018</b></p>

<hr style="height:5px;border:none" />

* Simple stuff: nodes & edges
* Drawing
* Weighted
* Drawing nodes, edges, and labels separately
* Exercise: simple network construction
* File formats
* Example networks
* Exercise: Large network construction
